In [3]:
import tensorflow as tf

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

def pipe(data, batch_size = 128, shuffle = False):
    dataset = tf.data.Dataset.from_tensor_slices(data)
    if shuffle:
        dataset = dataset.shuffle(buffer_size = batch_size * 10)
    dataset = dataset.batch(batch_size)
    #dataset = dataset.prefetch((batch_size * 2) + 1)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

(tr_x, tr_y), (te_x, te_y) = tf.keras.datasets.cifar10.load_data()

tr_x = tr_x * 1/255
te_x = te_x * 1/255

batch_size = 128

tr_data = pipe((tr_x, tr_y), batch_size = batch_size, shuffle = True)
te_data = pipe((te_x, te_y), batch_size = batch_size, shuffle = False)

In [14]:
import coatnet

model = coatnet.coatnet0(input_shape = (32, 32, 3), include_top = True)

'''flatten = tf.keras.layers.GlobalAveragePooling2D()(model.output)
drop_out = tf.keras.layers.Dropout(0.5)(flatten)
dense = tf.keras.layers.Dense(2048, activation = "relu")(drop_out)
prediction = tf.keras.layers.Dense(10, activation = "softmax", name = "prediction")(dense)
model = tf.keras.Model(model.input, prediction)'''

TypeError: Exception encountered when calling ConvTransformer.call().

[1mLayer.add_weight() got multiple values for argument 'shape'[0m

Arguments received by ConvTransformer.call():
  • args=('<KerasTensor shape=(None, 4, 4, 192), dtype=float32, sparse=False, name=keras_tensor_186>',)
  • kwargs=<class 'inspect._empty'>

In [3]:
loss = tf.keras.losses.sparse_categorical_crossentropy
opt = tf.keras.optimizers.Adam(1e-4)
metric = [tf.keras.metrics.sparse_categorical_accuracy]
model.compile(loss = loss, optimizer = opt, metrics = metric)

In [4]:
model.fit(tr_data, validation_data = te_data, epochs = 10)

Epoch 1/10
391/391 [==============================] - 122s 232ms/step - loss: 1.9192 - sparse_categorical_accuracy: 0.2442 - val_loss: 1.8568 - val_sparse_categorical_accuracy: 0.2575
Epoch 2/10
391/391 [==============================] - 81s 207ms/step - loss: 1.8239 - sparse_categorical_accuracy: 0.2806 - val_loss: 1.7438 - val_sparse_categorical_accuracy: 0.3051
Epoch 3/10
391/391 [==============================] - 81s 206ms/step - loss: 1.6805 - sparse_categorical_accuracy: 0.3433 - val_loss: 1.6522 - val_sparse_categorical_accuracy: 0.3601
Epoch 4/10
391/391 [==============================] - 81s 207ms/step - loss: 1.6476 - sparse_categorical_accuracy: 0.3630 - val_loss: 1.6518 - val_sparse_categorical_accuracy: 0.3702
Epoch 5/10
391/391 [==============================] - 80s 206ms/step - loss: 1.6377 - sparse_categorical_accuracy: 0.3659 - val_loss: 1.6750 - val_sparse_categorical_accuracy: 0.3463
Epoch 6/10
391/391 [==============================] - 81s 206ms/step - loss: 1.6530 

In [5]:
model.save_weights("model.h5")

In [6]:
model.load_weights("model.h5")

In [7]:
loss = tf.keras.losses.sparse_categorical_crossentropy
metric = [tf.keras.metrics.sparse_categorical_accuracy]
model.compile(loss = loss, metrics = metric)
model.evaluate(te_data)

79/79 [==============================] - 11s 72ms/step - loss: 1.4437 - sparse_categorical_accuracy: 0.4637


[1.4436622858047485, 0.46369999647140503]